In [78]:
import numpy as np
import pandas as pd
import matrix
import model
import analysis as an

In [82]:
def analysis(init, patterns, end_time):
    """sparsity_analysis.
    Generate simulation data to analyze how K_ac
    sparsity affects model dynamics.

    :param init:  inital abudance vector
    :param pattern: matrix patterns to test
    :param end_time: time steps to simulate
    """
    rows, n = [], len(init)
    fixation_times = []
    for pattern in patterns:
        curr_mat = matrix.pattern_matrix(pattern, n)
        
        # simulate model with given params
        t, N, S, E, P = model.simulate(curr_mat, init, end_time)
        # N, S are variables by time points matrices
        
        # Calculate statistics
        row = {"pattern": pattern}
        
        # Compute time to fixation
        row['fixation_time'] = len(N[0])
        
        # Compute growth rate
        row["growth_rate"] = an.growth_rate(N.T)
        
        N = np.apply_along_axis(an.normalize, 1, N.T).T
        
        # Compute skewness 
        row["skewness"] = an.skewness(N.T)
        
        # Compute richness
        richnessVals = [an.richness(i) for i in N.T]
        row["richness"] = richnessVals
        
        # Compute shannon index
        row["shannon index"] = an.shannon_index(N.T)
        
        # Add Bray Curtis distance metric array for each K_ac simulation
        brayCurtisMtx = np.identity(n)
        for i in range(n):
            for j in range(i + 1, n):
                brayCurtisMtx[i, j] = an.bray_curtis(N[i], N[j])
        row["bray curtis"] = brayCurtisMtx
        
        # store for easy conversion to a dataframe
        rows.append(row)
        
    return pd.DataFrame(rows)

In [83]:
patterns = ['null', 'ident', 'barbell', 'cycle', 'complete', 'star']

In [84]:
analysis([0.01, 0.002, 0.05, 0.03], patterns, 50)

,pattern,fixation_time,growth_rate,skewness,richness,shannon index,bray curtis
0,null,72,-0.007797,"[-1.1074895011665498, -1.0467941977219488, 1.1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[17.367679440430404, 5.99265609885249, 23.8603...","[[1.0, 0.6666666666666664, 0.6666666666666665,..."
1,ident,87,0.043022,"[-0.06234242320971703, 0.8596558093742553, -0....","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[26.14635302909244, 16.531130998999117, 30.635...","[[1.0, 0.3235311527260071, 0.39890767131291816..."
2,barbell,155,0.007589,"[-0.22698953257875842, 0.6120967659972473, 0.0...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[40.95947252560679, 5.84962787282245, 51.36674...","[[1.0, 0.9660479569735204, 0.47984313520464184..."
3,cycle,89,0.041799,"[0.40000501516497755, -0.4000050151649745, 0.4...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[23.436174446955157, 5.165418878799035, 24.584...","[[1.0, 0.8043762604489195, 0.6666666666666667,..."
4,complete,65,0.061861,"[-0.7361230101535051, -0.6140626453190065, 1.1...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[13.174259276078669, 4.270012829415719, 18.724...","[[1.0, 0.6744510241216142, 0.7658454918338498,..."
5,star,143,0.005508,"[0.6703797636271245, -0.6703797636270715, -0.6...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[18.254200860184675, 12.516615319044721, 44.99...","[[1.0, 0.43817174700132244, 0.855261660604453,..."
